In [2]:
!pip install fitz

In [1]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 77.9 MB/s eta 0:00:00


In [4]:
!pip install frontend

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.0 MB/s eta 0:00:00


In [17]:
# Install necessary libraries and create required directories
!pip install --upgrade pymupdf
!pip install pdfplumber
!pip install colorama
!mkdir static

import fitz
import pdfplumber
import os
from colorama import Fore, Style


# --- Helper Functions ---

def extract_page_content(pdf_path, page_num):
    """
    Fetch text and tables from a specific page of the PDF.
    Think of this as mining a treasure chest for data on the given page.
    """
    try:
        with pdfplumber.open(pdf_path) as pdf:
            if not (0 <= page_num < len(pdf.pages)):
                return None, None
            page = pdf.pages[page_num]
            text = page.extract_text()
            tables = page.extract_tables()
            return text, tables
    except Exception as e:
        print(Fore.RED + f"Error reading text and tables: {e}" + Style.RESET_ALL)
        return None, None


def extract_images(pdf_path, page_num, img_dir):
    """
    Extracts images from a specific page of the PDF and saves them locally.

    """
    try:
        if not os.path.exists(img_dir):
            os.makedirs(img_dir)
        images = []
        with fitz.open(pdf_path) as pdf:
            if not (0 <= page_num < len(pdf)):
                return None
            page = pdf[page_num]
            for i, img in enumerate(page.get_images(full=True)):
                xref = img[0]
                base_img = pdf.extract_image(xref)
                img_bytes = base_img["image"]
                img_filename = os.path.join(img_dir, f"page_{page_num + 1}_img{i + 1}.png")
                with open(img_filename, "wb") as img_file:
                    img_file.write(img_bytes)
                images.append(img_filename)
        return images
    except Exception as e:
        print(Fore.RED + f"Error extracting images: {e}" + Style.RESET_ALL)
        return []


def parse_user_query(query):
    """
    Analyzes the user query and extracts requested page numbers.
    Example: "Extract data from page 1 and page 3" -> [0, 2]
    """
    try:
        pages = [int(part.split()[0]) - 1 for part in query.split("page")[1:]]
        return pages
    except (IndexError, ValueError):
        raise ValueError("Oops! Couldn't understand the page numbers in your query. Please try again.")


def process_single_page(pdf_path, page_num, img_dir):
    """
    Gathers all the data(text, tables, and images) from a single page.
    """
    result = f"\n{Fore.YELLOW}--- Data from Page {page_num + 1} ---{Style.RESET_ALL}\n"

    # Extract text and tables
    text, tables = extract_page_content(pdf_path, page_num)
    if text is None:
        return f"{Fore.RED}Page {page_num + 1} does not exist in the PDF.{Style.RESET_ALL}\n"

    result += f"\n{Fore.GREEN}Text Content:{Style.RESET_ALL}\n{text or 'No text found on this page.'}\n"

    if tables:
        result += f"\n{Fore.CYAN}Tables Found:{Style.RESET_ALL}\n"
        for idx, table in enumerate(tables, 1):
            result += f"\nTable {idx}:\n"
            for row in table:
                result += " | ".join(str(cell) for cell in row) + "\n"
    else:
        result += f"\n{Fore.MAGENTA}No tables found on this page.{Style.RESET_ALL}\n"

    # Extract images
    images = extract_images(pdf_path, page_num, img_dir)
    if images:
        result += f"\n{Fore.BLUE}Images Extracted:{Style.RESET_ALL}\n" + "\n".join(images) + "\n"
    else:
        result += f"\n{Fore.MAGENTA}No images found on this page.{Style.RESET_ALL}\n"

    return result


def process_user_query(query, pdf_path, img_dir):
    """
    Processes the user query and retrieves data from all requested pages.
    """
    try:
        page_numbers = parse_user_query(query)
    except ValueError as e:
        return f"{Fore.RED}{e}{Style.RESET_ALL}"

    output = ""
    for page_num in page_numbers:
        output += process_single_page(pdf_path, page_num, img_dir)
    return output


# --- Main Execution ---

def main():

    PDF_PATH = "/content/sithafaltask1/sithafal file.pdf"
    IMG_DIR = "/content/output"

    print(Fore.BLUE + "Welcome to the PDF Data Extractor!" + Style.RESET_ALL)
    print("You can extract text, tables, and images from specific pages of a PDF.")
    print("For example, try: 'Extract data from page 1 and page 3'")

    user_query = input(Fore.YELLOW + "\nEnter your query: " + Style.RESET_ALL)
    output = process_user_query(user_query, PDF_PATH, IMG_DIR)

    print(Fore.GREEN + "\n--- Extraction Results ---\n" + Style.RESET_ALL)
    print(output)


if __name__ == "__main__":
    main()


mkdir: cannot create directory ‘static’: File exists
Welcome to the PDF Data Extractor!
You can extract text, tables, and images from specific pages of a PDF.
For example, try: 'Extract data from page 1 and page 3'

Enter your query: page 6

--- Extraction Results ---


--- Data from Page 6 ---

Text Content:
Table of Yearly U.S. GDP by
Industry (in millions of dollars)
Source: U.S. Bureau of Labor Statistics
Year 2010 2011 2012 2013 2014 2015
All Industries 26093515 27535971 28663246 29601191 30895407 31397023
Manufacturing 4992521 5581942 5841608 5953299 6047477 5829554
Finance,
Insurance, Real
4522451 4618678 4797313 5031881 5339678 5597018
Estate, Rental,
Leasing
Arts,
Entertainment,
Recreation, 964032 1015238 1076249 1120496 1189646 1283813
Accommodation,
and Food Service
Other 15614511 16320113 16948076 17495515 18318606 18686638

Tables Found:

Table 1:
Year | 2010 | 2011 | 2012 | 2013 | 2014 | 2015
All Industries | 26093515 | 27535971 | 28663246 | 29601191 | 30895407 | 31397023